# $\partial\textrm{SGP4}$ Autodiff Support

In this notebook, we show how to use the autodiff feature of $\partial\textrm{SGP4}$. Due to the fact that it is written in `pytorch`, it automatically supports automatic differentiation via `torch.autograd`. 

In this notebook, we show how these partial derivatives can be constructed: for more advanced examples on how to use these gradients for practical applications, see the tutorials on `state_transition_matrix_computation`, `covariance_propagation`, `graident_based_optimization`, `orbit_determination`.

In [1]:
import dsgp4
import torch

We create a TLE object

In [2]:
#as always, first, we create a TLE object:
tle=[]
tle.append('0 COSMOS 2251 DEB')
tle.append('1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996')
tle.append('2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320')
tle = dsgp4.tle.TLE(tle)
print(tle)

TLE(
0 COSMOS 2251 DEB
1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996
2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320
)


Now, as shown in the `tle_propagation` tutorial, we can propagate the TLE. However, instead of using the standard API, we require `torch.autograd` to record the operations w.r.t. the time.

## Partials with respect to time

Let's compute the partials of the $\partial \textrm{SGP4}$ output w.r.t. the propagation times

### Single TLEs

Let's first see the case of single TLEs, propagated at various future times:

In [3]:
#let's take a random tensor of 10 tsince elements, where we track the gradients:
tsince=torch.rand((10,),requires_grad=True)
#the state is then:
state_teme = dsgp4.propagate(tle,
                tsinces=tsince,
                initialized=False)
#now, we can see that the gradient is tracked:
print(state_teme)

tensor([[[ 1.3318e+03, -7.0047e+03,  2.4304e+01],
         [ 2.0168e+00,  4.0082e-01,  7.2028e+00]],

        [[ 1.3593e+03, -6.9985e+03,  1.2300e+02],
         [ 1.9965e+00,  5.0645e-01,  7.2017e+00]],

        [[ 1.3535e+03, -6.9999e+03,  1.0218e+02],
         [ 2.0008e+00,  4.8418e-01,  7.2021e+00]],

        [[ 1.3630e+03, -6.9976e+03,  1.3629e+02],
         [ 1.9937e+00,  5.2067e-01,  7.2014e+00]],

        [[ 1.3998e+03, -6.9865e+03,  2.7032e+02],
         [ 1.9654e+00,  6.6398e-01,  7.1973e+00]],

        [[ 1.4055e+03, -6.9846e+03,  2.9104e+02],
         [ 1.9609e+00,  6.8611e-01,  7.1964e+00]],

        [[ 1.3804e+03, -6.9927e+03,  1.9949e+02],
         [ 1.9804e+00,  5.8826e-01,  7.1998e+00]],

        [[ 1.3265e+03, -7.0058e+03,  5.4107e+00],
         [ 2.0206e+00,  3.8059e-01,  7.2029e+00]],

        [[ 1.4223e+03, -6.9784e+03,  3.5298e+02],
         [ 1.9475e+00,  7.5228e-01,  7.1933e+00]],

        [[ 1.4269e+03, -6.9766e+03,  3.6991e+02],
         [ 1.9438e+00,  7.7035e-

Let's now retrieve the partial derivatives of the SGP4 output w.r.t. time.

Since the state is position and velocity (i.e., $[x,y,z,v_x,v_y,v_z]$), these partials will be all the elements of type:
\begin{equation}
\dfrac{d \pmb{x}}{d t}=[\dfrac{dx}{dt}, \dfrac{dy}{dt}, \dfrac{dz}{dt}, \dfrac{d^2x}{dt^2}, \dfrac{d^2y}{dt^2}, \dfrac{d^2z}{dt^2}]^T=[v_x, v_y, v_z, \dfrac{dv_x}{dt}, \dfrac{dv_y}{dt}, \dfrac{dv_z}{dt}]^T
\end{equation}


```{note}
One thing to be careful about is that $\partial\textrm{SGP4}$, mirroring the original $\textrm{SGP4}$, takes the time in minutes, and returns the state in km and km/s. Hence, the derivatives will have dimensions coherent to these, and to return to SI, conversions have to be made.
```

In [4]:
partial_derivatives = torch.zeros_like(state_teme)
for i in [0,1]:
    for j in [0,1,2]:
        tsince.grad=None
        state_teme[:,i,j].backward(torch.ones_like(tsince),retain_graph=True)
        partial_derivatives[:,i,j] = tsince.grad

#let's print to screen the partials:
print(partial_derivatives)

tensor([[[ 1.2101e+02,  2.4049e+01,  4.3217e+02],
         [-8.7979e-02,  4.6273e-01, -1.6100e-03]],

        [[ 1.1979e+02,  3.0387e+01,  4.3210e+02],
         [-8.9791e-02,  4.6230e-01, -8.1460e-03]],

        [[ 1.2005e+02,  2.9051e+01,  4.3212e+02],
         [-8.9411e-02,  4.6240e-01, -6.7677e-03]],

        [[ 1.1962e+02,  3.1240e+01,  4.3209e+02],
         [-9.0034e-02,  4.6223e-01, -9.0261e-03]],

        [[ 1.1792e+02,  3.9839e+01,  4.3184e+02],
         [-9.2462e-02,  4.6147e-01, -1.7902e-02]],

        [[ 1.1766e+02,  4.1167e+01,  4.3178e+02],
         [-9.2834e-02,  4.6134e-01, -1.9273e-02]],

        [[ 1.1883e+02,  3.5296e+01,  4.3199e+02],
         [-9.1183e-02,  4.6190e-01, -1.3212e-02]],

        [[ 1.2124e+02,  2.2835e+01,  4.3217e+02],
         [-8.7630e-02,  4.6280e-01, -3.5867e-04]],

        [[ 1.1685e+02,  4.5137e+01,  4.3160e+02],
         [-9.3942e-02,  4.6091e-01, -2.3375e-02]],

        [[ 1.1663e+02,  4.6222e+01,  4.3154e+02],
         [-9.4244e-02,  4.6079e-

### Batch TLEs

Let's now see how it works for batch TLEs. The API is basically identical:

In [5]:
#we load 6 TLEs:
inp_file="""0 PSLV DEB
1 35350U 01049QJ  22068.76869562  .00000911  00000-0  24939-3 0  9998
2 35350  98.6033  64.7516 0074531  99.8340 261.1278 14.48029442457561
0 PSLV DEB *
1 35351U 01049QK  22066.70636923  .00002156  00000-0  63479-3 0  9999
2 35351  98.8179  29.5651 0005211  45.5944 314.5671 14.44732274457505
0 SL-18 DEB
1 35354U 93014BD  22068.76520028  .00021929  00000-0  20751-2 0  9995
2 35354  75.7302 100.7819 0059525 350.7978   9.2117 14.92216400847487
0 SL-18 DEB
1 35359U 93014BJ  22068.55187275  .00025514  00000-0  24908-2 0  9992
2 35359  75.7369 156.1582 0054843  50.5279 310.0745 14.91164684775759
0 SL-18 DEB
1 35360U 93014BK  22068.44021735  .00019061  00000-0  20292-2 0  9992
2 35360  75.7343 127.2487 0071107  32.5913 327.9635 14.86997880798827
0 METEOR 2-17 DEB
1 35364U 88005Y   22067.81503681  .00001147  00000-0  84240-3 0  9995
2 35364  82.5500  92.4124 0018834 303.2489 178.0638 13.94853833332534"""
lines=inp_file.splitlines()
#let's create the TLE objects
tles=[]
for i in range(0,len(lines),3):
    data=[]
    data.append(lines[i])
    data.append(lines[i+1])
    data.append(lines[i+2])
    tles.append(dsgp4.tle.TLE(data))
#we also create 9 random times, tracking the gradients:
tsinces=torch.rand((6,),requires_grad=True)

In [6]:
#let's propagate the batch:
state_teme = dsgp4.propagate_batch(tles,
                tsinces=tsinces,
                initialized=False)

Now, let's retrieve the partial of each TLE, at each propagated time, and store them into a Nx2x3 matrix:

In [7]:
#let's retrieve the partials w.r.t. time:
partial_derivatives = torch.zeros_like(state_teme)
for i in [0,1]:
    for j in [0,1,2]:
        tsinces.grad=None
        state_teme[:,i,j].backward(torch.ones_like(tsinces),retain_graph=True)
        partial_derivatives[:,i,j] = tsinces.grad

#let's print to screen the partials:
print(partial_derivatives)

tensor([[[ 5.3047e+01, -4.4699e+01,  4.4343e+02],
         [-2.0339e-01, -4.2612e-01, -1.4617e-02]],

        [[ 2.0159e+01, -6.7614e+01,  4.4352e+02],
         [-4.1173e-01, -2.3075e-01, -1.5824e-02]],

        [[-1.0683e+02, -3.9549e+01,  4.4202e+02],
         [ 9.8180e-02, -4.8835e-01, -2.0083e-02]],

        [[-3.3418e+01, -1.0775e+02,  4.4114e+02],
         [ 4.5454e-01, -1.9776e-01, -1.1247e-02]],

        [[-8.0926e+01, -7.9038e+01,  4.4180e+02],
         [ 3.0291e-01, -3.9315e-01, -1.2472e-02]],

        [[ 4.6984e+01, -3.6912e+02, -2.4033e+02],
         [ 3.8217e-02,  2.4712e-01, -3.7223e-01]]])


## Partials with respect to TLE parameters

Let's now tackle the case in which we are interested in the partials of the $\partial\textrm{SGP4}$ output w.r.t. the TLE parameters

### Single TLEs

We first tackle the case of single TLE, propagated at multiple times:

In this case, we want the Jacobian of the output state, w.r.t. the following TLE parameters $\textrm{TLE}=[n,e,i,\Omega,\omega,M,B^*,\dot{n},\ddot{n}]$, where:

* $n$ is the mean motion (also known as `no_kozai` in the original implementation) [rad/minute]; 
* $e$ is the eccentricity [-]; 
* $i$ is the inclination [rad]; 
* $\Omega$ is the right ascension of the ascending node [rad]; 
* $\omega$ is the argument of perigee [rad];
* $M$ is the mean anomaly [rad];
* $B^*$ is the Bstar parameter [1/earth radii]
* $\dot{n}$ mean motion first derivative [radians/$\textrm{minute}^2$]
* $\ddot{n}$ mean motion second derivative [radians/$\textrm{minute}^2$]

\begin{equation}
\dfrac{\partial \pmb{x}}{\partial \textrm{TLE}}=
\begin{bmatrix}
\frac{\partial x}{\partial B^*} & \frac{\partial x}{\partial \dot{n}} & \frac{\partial x}{\partial \ddot{n}} & \frac{\partial x}{\partial e} & \frac{\partial x}{\partial \omega} & \frac{\partial x}{\partial i} & \frac{\partial x}{\partial M} & \frac{\partial x}{\partial n} & \frac{\partial x}{\partial \Omega} \\
\\
\frac{\partial y}{\partial B^*} & \frac{\partial y}{\partial \dot{n}} & \frac{\partial y}{\partial \ddot{n}} & \frac{\partial y}{\partial e} & \frac{\partial y}{\partial \omega} & \frac{\partial y}{\partial i} & \frac{\partial y}{\partial M} & \frac{\partial y}{\partial n} & \frac{\partial y}{\partial \Omega} \\
\\
\frac{\partial z}{\partial B^*} & \frac{\partial z}{\partial \dot{n}} & \frac{\partial z}{\partial \ddot{n}} & \frac{\partial z}{\partial e} & \frac{\partial z}{\partial \omega} & \frac{\partial z}{\partial i} & \frac{\partial z}{\partial M} & \frac{\partial z}{\partial n} & \frac{\partial z}{\partial \Omega} \\
\\
\frac{\partial v_x}{\partial B^*} & \frac{\partial v_x}{\partial \dot{n}} & \frac{\partial v_x}{\partial \ddot{n}} & \frac{\partial v_x}{\partial e} & \frac{\partial v_x}{\partial \omega} & \frac{\partial v_x}{\partial i} & \frac{\partial v_x}{\partial M} & \frac{\partial v_x}{\partial n} & \frac{\partial v_x}{\partial \Omega} \\
\\
\frac{\partial v_y}{\partial B^*} & \frac{\partial v_y}{\partial \dot{n}} & \frac{\partial v_y}{\partial \ddot{n}} & \frac{\partial v_y}{\partial e} & \frac{\partial v_y}{\partial \omega} & \frac{\partial v_y}{\partial i} & \frac{\partial v_y}{\partial M} & \frac{\partial v_y}{\partial n} & \frac{\partial v_y}{\partial \Omega} \\
\\
\frac{\partial v_z}{\partial B^*} & \frac{\partial v_z}{\partial \dot{n}} & \frac{\partial v_z}{\partial \ddot{n}} & \frac{\partial v_z}{\partial e} & \frac{\partial v_z}{\partial \omega} & \frac{\partial v_z}{\partial i} & \frac{\partial v_z}{\partial M} & \frac{\partial v_z}{\partial n} & \frac{\partial v_z}{\partial \Omega} \\
\\
\frac{\partial \dot{n}}{\partial B^*} & \frac{\partial \dot{n}}{\partial \dot{n}} & \frac{\partial \dot{n}}{\partial \ddot{n}} & \frac{\partial \dot{n}}{\partial e} & \frac{\partial \dot{n}}{\partial \omega} & \frac{\partial \dot{n}}{\partial i} & \frac{\partial \dot{n}}{\partial M} & \frac{\partial \dot{n}}{\partial n} & \frac{\partial \dot{n}}{\partial \Omega} \\
\\
\frac{\partial \ddot{n}}{\partial B^*} & \frac{\partial \ddot{n}}{\partial \dot{n}} & \frac{\partial \ddot{n}}{\partial \ddot{n}} & \frac{\partial \ddot{n}}{\partial e} & \frac{\partial \ddot{n}}{\partial \omega} & \frac{\partial \ddot{n}}{\partial i} & \frac{\partial \ddot{n}}{\partial M} & \frac{\partial \ddot{n}}{\partial n} & \frac{\partial \ddot{n}}{\partial \Omega}
\end{bmatrix}
\end{equation}

In [14]:
#as always, first, we create a TLE object:
tle=[]
tle.append('0 COSMOS 2251 DEB')
tle.append('1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996')
tle.append('2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320')
tle = dsgp4.tle.TLE(tle)
print(tle)
tle_elements=dsgp4.initialize_tle(tle,with_grad=True)

TLE(
0 COSMOS 2251 DEB
1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996
2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320
)


In [173]:
#let's select 10 random times:
tsince=torch.rand((10,))
#and let's propagate:
state_teme=dsgp4.propagate(tle,tsince)

In [175]:
#now we can build the partial derivatives matrix, of shape Nx6x9 (N is the number of tsince elements, 6 is the number of elements in the state vector, and 9 is the number of elements in the TLE):
partial_derivatives = torch.zeros((len(tsince),6,9))
for k in range(len(tsince)):
    for i in range(6):
        tle_elements.grad=None
        state_teme[k].flatten()[i].backward(retain_graph=True)
        partial_derivatives[k,i,:] = tle_elements.grad
#let's print them to screen:
print(partial_derivatives)

tensor([[[-1.1153e-04,  0.0000e+00,  0.0000e+00,  9.5701e+02,  1.8798e+03,
          -2.2245e+02,  1.8945e+03, -1.3809e+04,  6.9905e+03],
         [-5.2772e-04,  0.0000e+00,  0.0000e+00,  6.4819e+03,  6.0225e+02,
          -4.5455e+01,  5.9124e+02,  7.4630e+04,  1.3877e+03],
         [-7.2626e-04,  0.0000e+00,  0.0000e+00,  7.5037e+03,  6.8538e+03,
           6.0420e+01,  6.8972e+03,  1.0543e+03,  0.0000e+00],
         [-1.0346e-06,  0.0000e+00,  0.0000e+00,  8.1737e-01, -1.4552e+00,
          -7.0701e+00, -1.4639e+00,  9.7349e+00, -6.1659e-01],
         [-6.5984e-07,  0.0000e+00,  0.0000e+00,  4.6502e+00,  7.3521e+00,
          -1.3395e+00,  7.3738e+00,  7.0547e+00,  1.9748e+00],
         [-3.9817e-06,  0.0000e+00,  0.0000e+00,  5.8403e+00, -2.2312e-01,
           2.0508e+00, -2.3897e-01,  3.8199e+01,  0.0000e+00]],

        [[-1.5407e-04,  0.0000e+00,  0.0000e+00,  9.6475e+02,  1.8656e+03,
          -2.9095e+02,  1.8803e+03, -1.3716e+04,  6.9841e+03],
         [-6.9627e-04,  0.0000e+

### Batch TLEs:

As for the time derivatives, the API stays practically identical:

In [2]:
#we load 6 TLEs:
inp_file="""0 PSLV DEB
1 35350U 01049QJ  22068.76869562  .00000911  00000-0  24939-3 0  9998
2 35350  98.6033  64.7516 0074531  99.8340 261.1278 14.48029442457561
0 PSLV DEB *
1 35351U 01049QK  22066.70636923  .00002156  00000-0  63479-3 0  9999
2 35351  98.8179  29.5651 0005211  45.5944 314.5671 14.44732274457505
0 SL-18 DEB
1 35354U 93014BD  22068.76520028  .00021929  00000-0  20751-2 0  9995
2 35354  75.7302 100.7819 0059525 350.7978   9.2117 14.92216400847487
0 SL-18 DEB
1 35359U 93014BJ  22068.55187275  .00025514  00000-0  24908-2 0  9992
2 35359  75.7369 156.1582 0054843  50.5279 310.0745 14.91164684775759
0 SL-18 DEB
1 35360U 93014BK  22068.44021735  .00019061  00000-0  20292-2 0  9992
2 35360  75.7343 127.2487 0071107  32.5913 327.9635 14.86997880798827
0 METEOR 2-17 DEB
1 35364U 88005Y   22067.81503681  .00001147  00000-0  84240-3 0  9995
2 35364  82.5500  92.4124 0018834 303.2489 178.0638 13.94853833332534"""
lines=inp_file.splitlines()
#let's create the TLE objects
tles=[]
for i in range(0,len(lines),3):
    data=[]
    data.append(lines[i])
    data.append(lines[i+1])
    data.append(lines[i+2])
    tles.append(dsgp4.tle.TLE(data))
#we also create 9 random times, tracking the gradients:
tsinces=torch.rand((6,),requires_grad=True)

In [3]:
#let's now initialize the TLEs, activating the gradient tracking for the TLE parameters:
tle_elements=dsgp4.initialize_tle(tles,with_grad=True)

In [6]:
#let's now propagate the batch of TLEs:
state_teme = dsgp4.propagate_batch(tles,tsinces)

Finally, we can build the matrix that contains the partial of the SGP4 output w.r.t. the TLE parameters, for each TLE:

In [11]:
#now we can build the partial derivatives matrix, of shape Nx6x9 (N is the number of tsince elements, 6 is the number of elements in the state vector, and 9 is the number of elements in the TLE):
partial_derivatives = torch.zeros((len(tsinces),6,9))
for k in range(len(tsinces)):
    for i in range(6):
        tle_elements[k].grad=None
        state_teme[k].flatten()[i].backward(retain_graph=True)
        partial_derivatives[k,i,:] = tle_elements[k].grad
#let's print them to screen:
print(partial_derivatives)

tensor([[[-3.1903e-04,  0.0000e+00,  0.0000e+00, -1.2766e+03,  8.4984e+02,
           2.3083e+02,  8.2557e+02, -3.1909e+04, -6.4186e+03],
         [-1.5375e-03,  0.0000e+00,  0.0000e+00,  2.2293e+03, -6.9541e+02,
          -1.1094e+02, -7.4145e+02, -6.8084e+04,  3.0698e+03],
         [ 2.4263e-03,  0.0000e+00,  0.0000e+00, -1.3907e+04,  7.0365e+03,
          -3.6377e+01,  7.0212e+03,  1.1937e+03,  0.0000e+00],
         [-1.0806e-06,  0.0000e+00,  0.0000e+00,  3.0909e+00, -3.2356e+00,
           6.6789e+00, -3.2261e+00,  2.7383e+00,  7.7973e-01],
         [-9.3565e-07,  0.0000e+00,  0.0000e+00,  6.7907e+00, -6.7461e+00,
          -3.1512e+00, -6.7454e+00, -7.9252e+00,  8.6752e-01],
         [-3.8113e-06,  0.0000e+00,  0.0000e+00, -6.6669e-01, -3.2455e-01,
          -1.1147e+00, -2.6965e-01,  3.8840e+01,  0.0000e+00]],

        [[-4.1279e-04,  0.0000e+00,  0.0000e+00, -4.8694e+03,  2.1476e+02,
           1.7600e+02,  2.1274e+02, -6.5490e+04, -3.4604e+03],
         [-4.2969e-04,  0.0000e+